In [ ]:
!nvidia-smi

Sun Jun 20 16:22:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!pip install kaggle -q
!kaggle datasets download -d aladdinpersson/flickr8kimagescaptions
!unzip -q flickr8kimagescaptions.zip

 99% 1.03G/1.04G [00:09<00:00, 158MB/s]
100% 1.04G/1.04G [00:09<00:00, 121MB/s]


In [ ]:
# !rm -r Image-Captioning/

In [ ]:
# # get the code for kaggle
# !git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = 'Image-Captioning/'
import sys
sys.path.append(py_files_path)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# !pip3 install nltk==3.6.2
# !python -m nltk.downloader wordnet
# import nltk
import nltk
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from torch.utils.tensorboard import SummaryWriter
from os import path as osp

In [ ]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 10  # number of epochs to train for (if early stopping is not triggered)
batch_size = 256
workers = 2
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder?
checkpoint = None  # path to checkpoint, None if none

In [ ]:
DATA_NAME = 'flickr8k_tb'

# local
# DATA_JSON_PATH = 'data.json'
# IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'
#colab
DATA_JSON_PATH = 'Image-Captioning/data.json'
IMGS_PATH = 'flickr8k/images/'

In [ ]:
# load vocab
vocab = build_vocab(DATA_JSON_PATH)
vocab_len = len(vocab)
vocab_len

100%|██████████| 40000/40000 [00:00<00:00, 264517.22it/s]


5089

In [ ]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

logger_dic = {
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder,
    'max_seq_length': 100,
    'vocab_size': vocab_len,
    'enocder': 'resnet101',
    'dropout': dropout,
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim 
    
}

In [ ]:
t_params

{'batch_size': 256,
 'data_name': 'flickr8k_tb',
 'decoder_lr': 0.0004,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 7,
 'fine_tune_encoder': False,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7f47dfc10c10>,
 'workers': 2}

In [ ]:
# experiment name
name = DATA_NAME + "exp_001"
# path
log_dir = 'experiments'

logger = SummaryWriter(log_dir=osp.join(log_dir, name))

In [ ]:
fit(t_params=t_params, m_params=m_params, logger=logger)

Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][0/118]	Batch Time 9.289 (9.289)	Data Load Time 3.644 (3.644)	Loss 9.4368 (9.4368)	Top-5 Accuracy 0.099 (0.099)
Epoch: [0][100/118]	Batch Time 2.444 (2.508)	Data Load Time 0.007 (0.040)	Loss 4.8555 (5.5165)	Top-5 Accuracy 53.882 (44.535)
Epoch train time 293.545 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/20]	Batch Time 5.785 (5.785)	Loss 5.4389 (5.4389)	Top-5 Accuracy 51.127 (51.127)	
----- Bleu-n Scores -----
1: 57.16301620580357
2: 31.49554775418225
3: 14.674086027137582
4: 6.729090558786988
-------------------------
----- METEOR Score -----


100%|██████████| 5000/5000 [00:11<00:00, 444.08it/s]


m: 0.25562311003422916

 * LOSS - 5.376, TOP-5 ACCURACY - 51.010, BLEU-4 - 6.729090558786988

Epoch validation time 65.610 (epoch_time.avg:.3f)
__________________________________________________
-------------------- Training --------------------
Epoch: [1][0/118]	Batch Time 6.138 (6.138)	Data Load Time 3.536 (3.536)	Loss 4.8740 (4.8740)	Top-5 Accuracy 52.718 (52.718)
Epoch: [1][100/118]	Batch Time 2.480 (2.481)	Data Load Time 0.001 (0.040)	Loss 4.5840 (4.6070)	Top-5 Accuracy 56.543 (56.997)
Epoch train time 289.841 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/20]	Batch Time 5.885 (5.885)	Loss 5.3430 (5.3430)	Top-5 Accuracy 53.383 (53.383)	
----- Bleu-n Scores -----


...

m: 0.31546484790486745

 * LOSS - 5.111, TOP-5 ACCURACY - 59.557, BLEU-4 - 13.642491782820764

Epoch validation time 64.491 (epoch_time.avg:.3f)
__________________________________________________
-------------------- Training --------------------
Epoch: [6][0/118]	Batch Time 6.132 (6.132)	Data Load Time 3.480 (3.480)	Loss 3.8940 (3.8940)	Top-5 Accuracy 66.279 (66.279)
Epoch: [6][100/118]	Batch Time 2.434 (2.495)	Data Load Time 0.010 (0.039)	Loss 3.7981 (3.7970)	Top-5 Accuracy 67.877 (67.370)
Epoch train time 291.759 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/20]	Batch Time 5.925 (5.925)	Loss 5.3135 (5.3135)	Top-5 Accuracy 58.007 (58.007)	
----- Bleu-n Scores -----


  1%|          | 37/5000 [00:00<00:13, 364.96it/s]

1: 65.38104481252968
2: 41.38574176181656
3: 24.062750301963334
4: 14.064188661020788
-------------------------
----- METEOR Score -----


100%|██████████| 5000/5000 [00:11<00:00, 447.79it/s]


m: 0.31977195257751523

 * LOSS - 5.041, TOP-5 ACCURACY - 59.702, BLEU-4 - 14.064188661020788

Epoch validation time 64.996 (epoch_time.avg:.3f)


In [ ]:
 checkpoint = load_checkpoint('/content/BEST_checkpoint_flickr8k_tb.pth.tar')

Loaded Checkpoint!!
Last Epoch: 6
Best Bleu-4: 14.064188661020788


In [ ]:
checkpoint_path = '/content/BEST_checkpoint_flickr8k_tb.pth.tar'

In [ ]:
fit(t_params=t_params, checkpoint=checkpoint_path ,m_params=m_params, logger=logger)

Loaded Checkpoint!!
Starting Epoch: 7
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [7][0/118]	Batch Time 6.166 (6.166)	Data Load Time 3.421 (3.421)	Loss 3.7866 (3.7866)	Top-5 Accuracy 67.299 (67.299)
Epoch: [7][100/118]	Batch Time 2.469 (2.496)	Data Load Time 0.001 (0.038)	Loss 3.6169 (3.7045)	Top-5 Accuracy 69.223 (68.690)
Epoch train time 292.085 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/20]	Batch Time 5.957 (5.957)	Loss 5.1431 (5.1431)	Top-5 Accuracy 59.713 (59.713)	
----- Bleu-n Scores -----


  1%|          | 32/5000 [00:00<00:15, 317.39it/s]

1: 65.35136442556798
2: 40.829227618565014
3: 23.396703139937802
4: 13.477235510478005
-------------------------
----- METEOR Score -----


100%|██████████| 5000/5000 [00:11<00:00, 429.50it/s]


m: 0.32751554085565837

 * LOSS - 5.050, TOP-5 ACCURACY - 60.375, BLEU-4 - 13.477235510478005

Epoch validation time 65.703 (epoch_time.avg:.3f)

Epochs since last improvement: (1,)
__________________________________________________
-------------------- Training --------------------
Epoch: [8][0/118]	Batch Time 6.002 (6.002)	Data Load Time 3.386 (3.386)	Loss 3.6323 (3.6323)	Top-5 Accuracy 70.360 (70.360)
Epoch: [8][100/118]	Batch Time 2.509 (2.507)	Data Load Time 0.007 (0.038)	Loss 3.6279 (3.6240)	Top-5 Accuracy 69.796 (69.765)
Epoch train time 292.830 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/20]	Batch Time 5.891 (5.891)	Loss 4.9571 (4.9571)	Top-5 Accuracy 61.376 (61.376)	
----- Bleu-n Scores -----


...

m: 0.32144561495957885

 * LOSS - 5.086, TOP-5 ACCURACY - 60.612, BLEU-4 - 13.910264657090575

Epoch validation time 65.872 (epoch_time.avg:.3f)

Epochs since last improvement: (2,)
__________________________________________________
-------------------- Training --------------------
Epoch: [9][0/118]	Batch Time 6.176 (6.176)	Data Load Time 3.618 (3.618)	Loss 3.5641 (3.5641)	Top-5 Accuracy 70.072 (70.072)
Epoch: [9][100/118]	Batch Time 2.467 (2.508)	Data Load Time 0.007 (0.041)	Loss 3.5851 (3.5457)	Top-5 Accuracy 70.481 (70.817)
Epoch train time 292.827 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/20]	Batch Time 5.882 (5.882)	Loss 5.6327 (5.6327)	Top-5 Accuracy 58.529 (58.529)	
----- Bleu-n Scores -----


  1%|          | 37/5000 [00:00<00:13, 361.82it/s]

1: 65.02147179979713
2: 40.79326500721345
3: 23.79599220505635
4: 14.041200188598557
-------------------------
----- METEOR Score -----


100%|██████████| 5000/5000 [00:12<00:00, 401.94it/s]


m: 0.3225425622768956

 * LOSS - 5.116, TOP-5 ACCURACY - 60.906, BLEU-4 - 14.041200188598557

Epoch validation time 65.665 (epoch_time.avg:.3f)

Epochs since last improvement: (3,)


In [ ]:
batch_size = 64
fine_tune_encoder = True
checkpoint_path = '/content/BEST_checkpoint_flickr8k_tb.pth.tar'
epochs = 20

t_params = {
    'data_name': DATA_NAME+'_finetune',
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr*.8,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_tb_finetune',
 'decoder_lr': 0.00032,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 20,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7f48955ce3d0>,
 'workers': 2}

In [ ]:
fit(t_params=t_params, checkpoint=checkpoint_path ,m_params=m_params, logger=logger)

Loaded Checkpoint!!
Starting Epoch: 7
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [7][0/469]	Batch Time 4.559 (4.559)	Data Load Time 1.040 (1.040)	Loss 3.7858 (3.7858)	Top-5 Accuracy 67.105 (67.105)
Epoch: [7][100/469]	Batch Time 1.283 (1.344)	Data Load Time 0.001 (0.011)	Loss 3.8368 (3.9083)	Top-5 Accuracy 67.036 (65.524)
Epoch: [7][200/469]	Batch Time 1.298 (1.332)	Data Load Time 0.001 (0.006)	Loss 3.8295 (3.8583)	Top-5 Accuracy 66.802 (66.374)
Epoch: [7][300/469]	Batch Time 1.325 (1.326)	Data Load Time 0.000 (0.005)	Loss 3.6381 (3.8268)	Top-5 Accuracy 69.721 (66.943)
Epoch: [7][400/469]	Batch Time 1.311 (1.323)	Data Load Time 0.001 (0.004)	Loss 3.6621 (3.8089)	Top-5 Accuracy 69.179 (67.265)
Epo

  1%|          | 33/5000 [00:00<00:15, 318.88it/s]

1: 67.00018921137398
2: 42.881750888701966
3: 25.892292195185686
4: 15.633913962034743
-------------------------
----- METEOR Score -----


100%|██████████| 5000/5000 [00:12<00:00, 414.99it/s]


m: 0.3378340797195308

 * LOSS - 5.037, TOP-5 ACCURACY - 61.684, BLEU-4 - 15.633913962034743

Epoch validation time 65.008 (epoch_time.avg:.3f)
__________________________________________________
-------------------- Training --------------------
Epoch: [8][0/469]	Batch Time 2.554 (2.554)	Data Load Time 1.129 (1.129)	Loss 3.5000 (3.5000)	Top-5 Accuracy 73.189 (73.189)
Epoch: [8][100/469]	Batch Time 1.312 (1.332)	Data Load Time 0.001 (0.013)	Loss 3.5291 (3.5750)	Top-5 Accuracy 70.175 (71.039)
Epoch: [8][200/469]	Batch Time 1.354 (1.325)	Data Load Time 0.001 (0.007)	Loss 3.6319 (3.5766)	Top-5 Accuracy 70.294 (70.929)
Epoch: [8][300/469]	Batch Time 1.299 (1.324)	Data Load Time 0.001 (0.005)	Loss 3.6524 (3.5751)	Top-5 Accuracy 70.201 (70.896)
Epoch: [8][400/469]	Batch Time 1.329 (1.323)	Data Load Time 0.001 (0.004)	Loss 3.5307 (3.5773)	Top-5 Accuracy 69.933 (70.885)
Epoch train time 620.007 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/79]	Batch Ti

...

  1%|          | 47/5000 [00:00<00:10, 463.87it/s]

1: 67.0097277326499
2: 43.36028209469986
3: 26.41766578125509
4: 16.195871539858164
-------------------------
----- METEOR Score -----


100%|██████████| 5000/5000 [00:11<00:00, 450.80it/s]


m: 0.34362699913690886

 * LOSS - 5.239, TOP-5 ACCURACY - 62.643, BLEU-4 - 16.195871539858164

Epoch validation time 62.849 (epoch_time.avg:.3f)

Epochs since last improvement: (1,)
__________________________________________________
-------------------- Training --------------------
Epoch: [11][0/469]	Batch Time 2.438 (2.438)	Data Load Time 1.067 (1.067)	Loss 3.1522 (3.1522)	Top-5 Accuracy 76.298 (76.298)
Epoch: [11][100/469]	Batch Time 1.293 (1.324)	Data Load Time 0.001 (0.012)	Loss 3.0582 (3.1495)	Top-5 Accuracy 80.318 (77.105)
Epoch: [11][200/469]	Batch Time 1.322 (1.320)	Data Load Time 0.000 (0.006)	Loss 3.2937 (3.1659)	Top-5 Accuracy 74.585 (76.893)


KeyboardInterrupt: ignored

In [ ]:
!mv BEST_checkpoint_flickr8k_tb_finetune.pth.tar /content/drive/MyDrive/ImageCaptioning

In [ ]:
!cp /content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_rmsprop.pth.tar /content/drive/MyDrive/weights/

In [ ]:
batch_size = 64
fine_tune_encoder = True
checkpoint = '/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_rmsprop.pth.tar'
epochs = 15

t_params = {
    'data_name': DATA_NAME+'_finetune',
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr*.8,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune',
 'decoder_lr': 0.00032,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 15,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7fcd545bc0d0>,
 'workers': 2}

In [ ]:
fit(t_params, checkpoint=checkpoint, m_params=m_params)

Loaded Checkpoint!!
Starting Epoch: 5
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [5][0/469]	Batch Time 1.736 (1.736)	Data Load Time 0.902 (0.902)	Loss 3.8268 (3.8268)	Top-5 Accuracy 68.293 (68.293)
Epoch: [5][100/469]	Batch Time 0.704 (0.720)	Data Load Time 0.000 (0.010)	Loss 4.1850 (4.0490)	Top-5 Accuracy 62.500 (63.931)
Epoch: [5][200/469]	Batch Time 0.705 (0.715)	Data Load Time 0.000 (0.006)	Loss 4.0673 (4.0145)	Top-5 Accuracy 64.947 (64.577)
Epoch: [5][300/469]	Batch Time 0.692 (0.713)	Data Load Time 0.000 (0.004)	Loss 3.8470 (3.9893)	Top-5 Accuracy 67.878 (64.951)
Epoch: [5][400/469]	Batch Time 0.715 (0.712)	Data Load Time 0.000 (0.004)	Loss 3.8155 (3.9740)	Top-5 Accuracy 68.421 (65.176)
---

KeyboardInterrupt: ignored

In [ ]:
!cp /content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune.pth.tar /content/drive/MyDrive/weights/

In [ ]:
checkpoint = load_checkpoint('/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune.pth.tar')

Loaded Checkpoint!!
Last Epoch: 12
Best Bleu-4: 0.15974322033502095


In [ ]:
batch_size = 64
fine_tune_encoder = True
checkpoint = '/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune.pth.tar'
epochs = 15

t_params = {
    'data_name': DATA_NAME+'_finetune_new_lr',
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr*.8,
    'encoder_lr': encoder_lr*.8,
    'fine_tune_encoder': fine_tune_encoder
}
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune_new_lr',
 'decoder_lr': 0.00032,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 8e-05,
 'epochs': 15,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7fcd545bc0d0>,
 'workers': 2}

In [ ]:
fit(t_params, checkpoint=checkpoint, m_params=m_params)

Loaded Checkpoint!!
Starting Epoch: 13
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [13][0/469]	Batch Time 1.702 (1.702)	Data Load Time 0.922 (0.922)	Loss 2.9290 (2.9290)	Top-5 Accuracy 79.778 (79.778)
Epoch: [13][100/469]	Batch Time 0.711 (0.723)	Data Load Time 0.000 (0.011)	Loss 2.7991 (2.7624)	Top-5 Accuracy 81.139 (82.264)
Epoch: [13][200/469]	Batch Time 0.719 (0.717)	Data Load Time 0.000 (0.006)	Loss 2.9192 (2.7909)	Top-5 Accuracy 80.526 (81.872)
Epoch: [13][300/469]	Batch Time 0.743 (0.715)	Data Load Time 0.012 (0.005)	Loss 2.8699 (2.8093)	Top-5 Accuracy 80.472 (81.621)
Epoch: [13][400/469]	Batch Time 0.711 (0.713)	Data Load Time 0.000 (0.004)	Loss 2.8068 (2.8186)	Top-5 Accuracy 82.283 (81.48

In [ ]:
checkpoint = load_checkpoint('/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune.pth.tar')

Loaded Checkpoint!!
Last Epoch: 5
Best Bleu-4: 0.15255731663785566


In [ ]:
checkpoint = load_checkpoint('/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam.pth.tar')

Loaded Checkpoint!!
Last Epoch: 4
Best Bleu-4: 0.1466164024634058
